# **Tugas Pertemuan 1 - Crawling berita online**

Crawling berita online detik.com

Crawling data atau web crawling adalah proses ekstraksi informasi dari situs web dengan menggunakan program komputer. Tujuan utama dari web crawling adalah untuk mengumpulkan data dari berbagai sumber di internet.
Terdapat berbagai macam library yang dapat digunakan untuk crawling data. Library yang digunakan untuk crawling berita detik.com yaitu Beautiful Soup dan Requests. Beautiful Soup Digunakan untuk mengekstrak informasi dari HTML dan Request digunakan untuk mengirim permintaan HTTP dan mengakses halaman web.


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if content_element:
        # Hapus elemen yang berisi daftar isi
        for daftar_isi in ["collapsible"]:
            unwanted = content_element.find("div", id=daftar_isi)
            if unwanted:
                unwanted.decompose()

        # Hapus elemen yang berisi tag
        for tag_class in ["aevp", "detail__body-tag mgt-16"]:
            unwanted = content_element.find_all("div", class_=tag_class)
            for el in unwanted:
                el.decompose()

        # Hapus elemen yang berisi link sisipan
        link_sisip = content_element.find_all("table", class_="linksisip")
        for table in link_sisip:
            table.decompose()

        # Hapus elemen paragraf dan span dengan class 'para_caption'
        unwanted_paragraphs = content_element.find_all(["p", "span"], class_="para_caption")
        for para in unwanted_paragraphs:
            para.decompose()

        # Kembalikan teks yang tersisa
        return content_element.get_text(separator=' ', strip=True).strip()

    return "Content Not Found"


Selanjutnya kumpulkan data berita dari website Detik.com dan simpan dalam format CSV.

- Proses pengambilan data dengan fungsi get_data  digunakan untuk mengambil artikel  dari halaman yang ditentukan. Kode ini bertujuan crawl URL, mem-parsing HTML untuk menemukan artikel, dan kemudian mengambil detail artikel (judul, tanggal, isi berita, kategori) dari setiap tautan artikel yang ditemukan.

- Pembersihan Konten/data menggunakan fungsi clean_content untuk menghapus elemen yang tidak di inginkan.

- Save Data artikel (judul, tanggal, isi berita, kategori) disimpan dalam daftar list dan kemudian dikumpulkan dalam DataFrame Pandas.

In [6]:
# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len([k for k in kategori_list if k == kategori]) >= min_articles_per_category:
            return  # Menghentikan proses jika jumlah artikel sudah mencapai minimum yang diinginkan

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        # Bersihkan konten menggunakan fungsi clean_content
        content = clean_content(content_element)


        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        # url_list.append(link)
        kategori_list.append(kategori)

        if len(judul) <= 10:
          print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = [
    "https://sport.detik.com/indeks",
    "https://food.detik.com/indeks",
]
categories = [
    "Olahraga",
    "Makanan",
]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 50

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}/{page}"
        get_data(url, category, min_articles_per_category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman
        page += 1

# Membuat dataframe dari list data
df = pd.DataFrame({"judul": judul, "tanggal": tanggal, "isi": isi, "kategori": kategori_list})

# Menyimpan dataframe ke file csv
df.to_csv("data_berita_detik.csv", index=False)

# Menampilkan dataframe
# print(df)

Sulit Kejar Marc Marquez, Bagnaia Belum Fit 100 Persen
Potret Pelari Maraton Uganda yang Tewas Dibakar Pacar
Jokowi Resmikan Pembangunan-Renovasi Venue PON Aceh Senilai Rp 811 M
Dua Kali Ganti Motor Bikin Jorge Martin Gigit Jari di Misano
Menangi MotoGP San Marino di Luar Dugaan Marc Marquez
Paralimpiade Paris 2024 Usai, Indonesia Cetak Rekor 14 Medali
Meriahnya Penutupan Paralimpiade 2024
Jorge Martin Gagal Podium: Itu Salahku!
Jannik Sinner Rebut Titel Juara US Open 2024
Jika Terpilih Jadi Ketum Kormi, Ini Tiga Rencana Adil Hakim


Data yang diambil terdiri dari :
- Judul
- Isi berita
- Tanggal
- Kategori

Pada tahap ini menampilkan data tersebut dan diambil berdasarkan kategori
***Kesehatan*** dan ***Kuliner*** dan hanya mengambil 100 sampel data crawl

Berikut untuk 5 sampel data berdasarkan kategori ***Kesehatan***

In [7]:
df=pd.read_csv("data_berita_detik.csv")
df.head(5)

,judul,tanggal,isi,kategori
0,"Sulit Kejar Marc Marquez, Bagnaia Belum Fit 10...","Senin, 09 Sep 2024 14:30 WIB",Jakarta - Marc Marquez jadi kampiun di MotoGP ...,Olahraga
1,Potret Pelari Maraton Uganda yang Tewas Dibaka...,"Senin, 09 Sep 2024 13:30 WIB",Uganda - Pelari maraton Olimpiade Uganda Rebec...,Olahraga
2,Jokowi Resmikan Pembangunan-Renovasi Venue PON...,"3,387 Views | Senin, 09 Sep 2024 12:03 WIB",Presiden Joko Widodo ( Jokowi ) meresmikan pem...,Olahraga
3,Dua Kali Ganti Motor Bikin Jorge Martin Gigit ...,"Senin, 09 Sep 2024 13:15 WIB",Jakarta - Jorge Martin cuma finis ke-15 di Mot...,Olahraga
4,Menangi MotoGP San Marino di Luar Dugaan Marc ...,"3,620 Views | Senin, 09 Sep 2024 12:08 WIB",Marc Marquez tak menduga dia bakal memenangi M...,Olahraga


Berikut untuk 5 sampel data berdasarkan kategori ***Kuliner***

In [8]:
df.tail(5)

,judul,tanggal,isi,kategori
95,Nyeleneh! Pria Ini Makan Indomie Goreng Campur...,"Sabtu, 07 Sep 2024 17:00 WIB",Jakarta - Indomie goreng biasanya dinikmati de...,Makanan
96,Keren! Seniman Makanan Asal Jepang Ini Ubah Su...,"Sabtu, 07 Sep 2024 16:00 WIB","Jakarta - Seniman makanan asal Jepang, Takayo ...",Makanan
97,Paus Fransiskus Santap Parpadelle hingga Cicip...,"Sabtu, 07 Sep 2024 15:00 WIB","Jakarta - Selama di Jakarta, Paus Fransiskus d...",Makanan
98,Keren! Restoran McDonald's Ini Berkonsep 'Fine...,"Sabtu, 07 Sep 2024 14:00 WIB",Jakarta - Berbeda dengan konsep restoran cepat...,Makanan
99,Title Not Found,Date Not Found,Content Not Found,Makanan
